## Analyzing cropland mapping predictions

This notebook visualizes and analyzes the outputs of Unet model's cropland predictions on PlanetScope NICFI basemap data.

## Setup

Install rasterio and connect Drive folder used to run analysis

In [ ]:
!pip install rasterio

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

### Libraries

In [ ]:
import os
import re

import numpy as np
import numpy.ma as ma
import pandas as pd

import matplotlib.pyplot as plt
import cv2
import gdal
import rasterio
from rasterio.plot import show
from rasterio.plot import reshape_as_raster, reshape_as_image

# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Magic keywords for Ipython
%load_ext autoreload
%autoreload 2
%matplotlib inline

## Paths

In [ ]:
config = {
    "working_dir" : "/content/gdrive/MyDrive/working_folder",
    "out_dir" : "refine15",
    
    "root_dir" : 
        "/content/gdrive/MyDrive/teaching/geog287387/data/fieldmapping/",
    "catalog_train_fn" : "catalog_ghana_ecaas_ejura_tain.csv",
    "catalog_pred_fn" : "catalog_predict_nicfi_retiled_ejura_tain_2020-11.csv"
}

# Visualize

## Predictions

In [ ]:
pred_path = f'{config["working_dir"]}/{config["out_dir"]}/Inference_output'
preds = [f'{pred_path}/{file}' for file in os.listdir(pred_path)]
score_maps = [rasterio.open(p) for p in preds]

## Images

To get the images corresponding to the predictions, we have to use the prediction catalog and the name of the predictions. 

In [ ]:
pred_cat = pd.read_csv(f'{config["root_dir"]}/{config["catalog_pred_fn"]}')
pred_cat

In [ ]:
colrow = [re.sub("class*.*score_c|_r|.tif", "", os.path.basename(p)) \
          for p in preds]

In [ ]:
pred_cat["tile_col"].map(str) + pred_cat["tile_row"].map(str)

Fortunately, the images are in the same order in the catalog as the predictions are listed, so we can do one to one comparisons. 

In [ ]:
images = [rasterio.open(f'{config["root_dir"]}{img["dir_os"]}') \
         for row, img in pred_cat.iterrows()]

### Function to rescale image for plotting

In [ ]:
def rescale_image(image, bands=(0, 1, 2, 3)):
    img = reshape_as_image(image.read())[:,:,bands]
    max_vals = [img[:, :, band].max() for band in range(img.shape[-1])]
    img = img.astype('float64')
    for band in bands:
        band_vals = img[:, :, band]
        img[:, :, band] = band_vals / max_vals[band]

    return img

### Function to plot side by side images

In [ ]:
def image_plotter(image, prediction, fsize):

    fig, ax_arr = plt.subplots(1, 2, sharex=True, sharey=True, figsize=fsize)
    ax1, ax2 = ax_arr.ravel()
    
    ax1.imshow(image) 
    ax1.set_title('Original  image')

    ax2.imshow(prediction)
    ax2.set_title('Score map')

    for ax in ax_arr.ravel():
        ax.set_axis_off()

    plt.tight_layout()
    plt.show()

In [ ]:
img_rescaled = rescale_image(images[0])
score = score_maps[0].read()[0]

In [ ]:
image_plotter(img_rescaled[:,:,(3,2,1)], score, fsize = (10, 10))

In [ ]:
cropland = np.ma.masked_where(score_mask < 50, score_mask)